In [ ]:
#download dependencies 
!wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/kinematics.py
!wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/circstat.py
!wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/processing.py
!wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/skeleton.py

In [ ]:
%pip install scikit-video

import sys, os, cv2, glob, json, gc
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
import json
import numpy as np
import pandas as pd
import itertools
from itertools import chain
from moviepy.editor import VideoFileClip
import skvideo.io
from tqdm import tqdm
import circstat as CS
import scipy as sc
import math

from processing import * 
from kinematics import *
from skeleton import *